In [1]:
import xarray as xr
import rioxarray
import pandas as pd
import os
from pathlib import Path
import numpy as np

root = Path.cwd()

In [2]:
epsg = 26919
site_name = 'BART'
year = '2019'

b = xr.open_dataarray(root / 'sentinel_data' / f'{year}_{site_name}.nc')
b = b.rio.write_crs(epsg).rio.set_spatial_dims(x_dim="x",y_dim="y",).rio.write_coordinate_system()

In [ ]:
# select spring, summer, fall and winter images
# spring: 5/8 & 6/7
# summer: 8/1, 8/6 & 8/26
# fall: 10/6, 10/15 & 11/4

b.isel(band=4).plot(col='time',col_wrap=4)

In [17]:
# spring: 5/8 & 6/7  [6,7]
# summer: 8/1, 8/6 & 8/26    [14,15,16]
# fall: 10/5, 10/15 & 11/4   [19,20,21]

In [41]:
# get median seasonal values, drop time dimension, add prefix to band names
def get_season_image(a,season,indices,num_months):
    a = a.isel(time=slice(indices[0],indices[1])).resample(time=num_months).median()
    a = a.squeeze('time').reset_coords('time',drop=True)
    
    return a.assign_coords({'band':[f'{x}_{season}' for x in a.band.values]})

spring = get_season_image(a=b, season='spring', indices=[6,8],num_months='2MS')
summer = get_season_image(a=b, season='summer', indices=[14,17],num_months='3MS')
fall = get_season_image(a=b, season='fall', indices=[19,22],num_months='3MS')


In [43]:
season_bands = xr.concat([spring,summer,fall],dim='band')

In [ ]:
# calculate seasonal evi, lswi, slavi, psri

def remove_outliers(a,norm=True):
    a = a.where(np.isfinite(a),np.nan)
    if norm == True:
        a = a.where((a >= -1) & (a <= 1))
    else:
        a = a.where((a >= 0) & (a <= 8))
    return a

array_list = [spring,summer,fall]
season_list = ['spring','summer','fall']
to_concat = list()
for i in range(0,3):
    blue = array_list[i].sel(band=f'B02_{season_list[i]}')
    red = array_list[i].sel(band=f'B04_{season_list[i]}')
    nir = array_list[i].sel(band=f'B8A_{season_list[i]}')
    sw1 = array_list[i].sel(band=f'B11_{season_list[i]}')
    sw2 = array_list[i].sel(band=f'B12_{season_list[i]}')
    re2 = array_list[i].sel(band=f'B06_{season_list[i]}')


    evi = 2.5 * ((nir - red) / (nir + 6 * red - 7.5 * blue + 1)).expand_dims({'band':[f'evi_{season_list[i]}']}) # range -1 - 1
    evi = remove_outliers(evi)

    lswi = (nir - sw1)/(nir + sw1).expand_dims({'band':[f'lswi_{season_list[i]}']}) # range -1 to 1
    lswi = remove_outliers(lswi)

    slavi = nir/(red + sw2).expand_dims({'band':[f'slavi_{season_list[i]}']}) # range 0 - 8
    slavi = remove_outliers(slavi,norm=False)

    psri = (red - blue)/re2.expand_dims({'band':[f'psri_{season_list[i]}']}) # range -1 to 1
    psri = remove_outliers(psri)

    season_vi = xr.concat([evi,lswi,slavi,psri],dim='band')

    to_concat.append(season_vi)

r = xr.concat(to_concat,dim='band')





In [75]:
# summer - fall & summer - spring for evi, lswi, slavi, and psri
def get_season_differences(band,season):
    d = r.sel(band=f'{band}_summer') - r.sel(band=f'{band}_{season}')
    return d.expand_dims({'band':[f'{band}_{season}_diff']})

to_concat = list()
for vi in ['evi', 'lswi', 'slavi','psri']:
    for s in ['spring','fall']:
        t = get_season_differences(vi,s)
        to_concat.append(t)

m1 = xr.concat(to_concat,dim='band')
# evi spring
# evi fall

In [78]:
final = xr.concat([m1,r],dim='band')

In [79]:
final.to_netcdf(root / 'sentinel_data' / 'bartlett_stack.nc')

In [ ]:
# to do: 
    # PCA on final stack
    # mnf transformation
    # select endmembers
    # check endmembers using ppi
    # mtmf